## Luftdata med kartor

Simon Winter, Infontology http://infontology.org



I den här lektionen ska vi se hur man kan hitta data för luftföroreningar från det nätverk av sammankopplade sensorer som finns på Luftdata.se

In [2]:
import requests
import pandas as pd
import gmaps
from credentials import key

import gpxpy
import gpxpy.gpx

In [40]:
# https://console.developers.google.com/apis/dashboard?project=natural-furnace-264420
gmaps.configure(api_key=key)
HOME = (55.589957, 12.943115)

In [4]:
latitude = '55.611437'
longitude = '12.994264'
distance = '150' # Avstånd i kilometer

Med hjälp av koordinaterna så skapar vi urlen. Lämna tomma platser inom {} i textsträngen, och ge variabelnamnen i format-parentesen

In [5]:
url = 'http://api.luftdaten.info/v1/filter/area={},{},{}'.format(latitude, longitude, distance)
url ## Om man ger ett variabelnamn på sista raden av en cell skrivs värdet ut. Det går också att göra print(url)

'http://api.luftdaten.info/v1/filter/area=55.611437,12.994264,150'

Requests-biblioteket gör det lätt att hantera frågor till servern. 

In [6]:
response = requests.get(url)

In [7]:
sensors = response.json()
number_of_sensors = len(sensors)
number_of_sensors

141

Om number_of_sensors är större än 0 så finns det sensorer i listan. Då kan man få fram sensorvärdena på olika sätt. Det första värdet ligger alltid i sensors[0]

In [8]:
sens2 = pd.DataFrame(columns=['latitude', 'longitude'])

In [9]:
sens2

,latitude,longitude


In [10]:
def create_sensordict (sensor):
    sensordict = {}
    sensordict['longitude'] = sensor['location']['longitude']
    sensordict['latitude'] = sensor['location']['latitude']
    for stype in sensor['sensordatavalues']:
        sensordict[stype['value_type']] = stype['value']
    return sensordict

In [11]:
for sensor in sensors:
    sens2 = sens2.append (create_sensordict(sensor), ignore_index=True)

    

In [12]:
sens2

,latitude,longitude,humidity,temperature,P1,P2,pressure,pressure_at_sealevel
0,56.144,13.394,99.90,6.10,NaN,NaN,NaN,NaN
1,56.144,13.394,NaN,NaN,17.27,8.37,NaN,NaN
2,55.728,13.168,99.90,6.70,NaN,NaN,NaN,NaN
3,55.732,13.172,NaN,NaN,2.10,2.10,NaN,NaN
4,55.728,13.168,NaN,NaN,40.60,9.77,NaN,NaN
...,...,...,...,...,...,...,...,...
136,55.61,12.1,0.00,6.00,NaN,NaN,100234.66,100821.08
137,55.61,12.1,NaN,NaN,9.13,4.80,NaN,NaN
138,55.898,14.094,NaN,NaN,21.67,10.53,NaN,NaN
139,55.728,13.168,99.90,6.70,NaN,NaN,NaN,NaN


In [13]:
sens2 = sens2.astype('float')

In [14]:
sens2.dtypes

latitude                float64
longitude               float64
humidity                float64
temperature             float64
P1                      float64
P2                      float64
pressure                float64
pressure_at_sealevel    float64
dtype: object

In [15]:
humid_values = sens2[sens2['humidity'].notnull()]

In [16]:
locations = humid_values[['latitude', 'longitude']]
weights = humid_values['humidity']

In [39]:
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(point_list)
fig.add_layer(marker_layer)
fig



Figure(layout=FigureLayout(height='420px'))

In [18]:
value_dict = humid_values.to_dict('index')

In [19]:
value_dict.values()

dict_values([{'latitude': 56.144, 'longitude': 13.394, 'humidity': 99.9, 'temperature': 6.1, 'P1': nan, 'P2': nan, 'pressure': nan, 'pressure_at_sealevel': nan}, {'latitude': 55.728, 'longitude': 13.168, 'humidity': 99.9, 'temperature': 6.7, 'P1': nan, 'P2': nan, 'pressure': nan, 'pressure_at_sealevel': nan}, {'latitude': 55.898, 'longitude': 14.094, 'humidity': 99.9, 'temperature': 5.7, 'P1': nan, 'P2': nan, 'pressure': nan, 'pressure_at_sealevel': nan}, {'latitude': 55.61, 'longitude': 12.1, 'humidity': 0.0, 'temperature': 6.1, 'P1': nan, 'P2': nan, 'pressure': 100243.75, 'pressure_at_sealevel': 100830.02}, {'latitude': 54.518, 'longitude': 13.656, 'humidity': 91.2, 'temperature': 10.4, 'P1': nan, 'P2': nan, 'pressure': nan, 'pressure_at_sealevel': nan}, {'latitude': 56.07, 'longitude': 12.698, 'humidity': 99.9, 'temperature': 6.8, 'P1': nan, 'P2': nan, 'pressure': nan, 'pressure_at_sealevel': nan}, {'latitude': 55.722, 'longitude': 13.202, 'humidity': 99.9, 'temperature': 7.9, 'P1':

In [20]:
data = humid_values.to_dict('index').values()
#data

In [21]:

plant_locations = [(plant['latitude'], plant['longitude']) for plant in data] 
info_box_template = """
<dl>
<dt>Fuktighet</dt><dd>{humidity}</dd>
<dt>Lufttryck</dt><dd>{pressure}</dd>
</dl>
"""

plant_info = [info_box_template.format(**plant) for plant in data]
marker_layer = gmaps.marker_layer(plant_locations, info_box_content=plant_info)

In [22]:
fig = gmaps.figure()

fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Parsing an existing file:
# -------------------------

with open('min-etapp.gpx') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

In [36]:
len(gpx.tracks[0].segments[0].points)

1220

In [38]:
point_list = [(point.latitude, point.longitude) for point in gpx.tracks[0].segments[0].points]
point_list

[(56.08004340824433, 13.123206823376563),
 (56.08004340824432, 13.123206823376563),
 (56.08015507461923, 13.123286823877152),
 (56.0802717409943, 13.123321824377967),
 (56.08035174122221, 13.123405157319326),
 (56.08045340765238, 13.123455157725402),
 (56.08054674144187, 13.123513490274377),
 (56.080640075058234, 13.123580156454409),
 (56.08072174169309, 13.123660156508663),
 (56.08080674184227, 13.123723489585245),
 (56.08091340824089, 13.123806823378686),
 (56.08100340820117, 13.123885156779806),
 (56.08110340864946, 13.12393515600911),
 (56.081195075237055, 13.123965156144664),
 (56.081313408672266, 13.124041822635718),
 (56.081398408325605, 13.124101823231172),
 (56.08149007477156, 13.124190156943428),
 (56.08156674157159, 13.124280156713667),
 (56.08164674177589, 13.124366823028708),
 (56.081743408213995, 13.124403490087957),
 (56.08184007420349, 13.124446824584993),
 (56.08193007408515, 13.124516824788088),
 (56.08203007385659, 13.124556825180775),
 (56.08212007381693, 13.1246051

In [26]:
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            point_list = []

Point at (56.08004340824433,13.123206823376563) -> None
Point at (56.08004340824432,13.123206823376563) -> None
Point at (56.08015507461923,13.123286823877152) -> None
Point at (56.0802717409943,13.123321824377967) -> None
Point at (56.08035174122221,13.123405157319326) -> None
Point at (56.08045340765238,13.123455157725402) -> None
Point at (56.08054674144187,13.123513490274377) -> None
Point at (56.080640075058234,13.123580156454409) -> None
Point at (56.08072174169309,13.123660156508663) -> None
Point at (56.08080674184227,13.123723489585245) -> None
Point at (56.08091340824089,13.123806823378686) -> None
Point at (56.08100340820117,13.123885156779806) -> None
Point at (56.08110340864946,13.12393515600911) -> None
Point at (56.081195075237055,13.123965156144664) -> None
Point at (56.081313408672266,13.124041822635718) -> None
Point at (56.081398408325605,13.124101823231172) -> None
Point at (56.08149007477156,13.124190156943428) -> None
Point at (56.08156674157159,13.124280156713667

In [27]:
for waypoint in gpx.waypoints:
    print('waypoint {0} -> ({1},{2})'.format(waypoint.name, waypoint.latitude, waypoint.longitude))

In [29]:
for route in gpx.routes:
    print('Route:')
    for point in route.points:
        print('Point at ({0},{1}) -> {2}'.format(point.latitude, point.longitude, point.elevation))

In [30]:
# There are many more utility methods and functions:
# You can manipulate/add/remove tracks, segments, points, waypoints and routes and
# get the GPX XML file from the resulting object:

print('GPX:', gpx.to_xml())

GPX: <?xml version="1.0" encoding="UTF-8"?>
<gpx xmlns="http://www.topografix.com/GPX/1/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.topografix.com/GPX/1/1 http://www.topografix.com/GPX/1/1/gpx.xsd" version="1.1" creator="togpx">
  <trk>
    <name>min-etapp</name>
    <desc>name=min-etapp</desc>
    <trkseg>
      <trkpt lat="56.08004340824433" lon="13.123206823376563">
      </trkpt>
      <trkpt lat="56.08004340824432" lon="13.123206823376563">
      </trkpt>
      <trkpt lat="56.08015507461923" lon="13.123286823877152">
      </trkpt>
      <trkpt lat="56.0802717409943" lon="13.123321824377967">
      </trkpt>
      <trkpt lat="56.08035174122221" lon="13.123405157319326">
      </trkpt>
      <trkpt lat="56.08045340765238" lon="13.123455157725402">
      </trkpt>
      <trkpt lat="56.08054674144187" lon="13.123513490274377">
      </trkpt>
      <trkpt lat="56.080640075058234" lon="13.123580156454409">
      </trkpt>
      <trkpt lat="56.08

In [43]:
fig = gmaps.figure()
geneva2zurich_via_montreux = gmaps.directions_layer(
        point_list[0], HOME, travel_mode='TRANSIT')
point_layer = gmaps.marker_layer([point_list[0], HOME])
fig.add_layer(point_layer)
fig.add_layer(geneva2zurich_via_montreux)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
sensors[0]

{'sampling_rate': None,
 'location': {'country': 'SE',
  'altitude': '8.2',
  'id': 8180,
  'indoor': 0,
  'exact_location': 0,
  'longitude': '13.024',
  'latitude': '55.606'},
 'sensordatavalues': [{'value_type': 'temperature',
   'id': 12788794817,
   'value': '7.86'},
  {'value_type': 'pressure', 'id': 12788794818, 'value': '101336.69'},
  {'value_type': 'humidity', 'id': 12788794820, 'value': '78.45'},
  {'value_type': 'pressure_at_sealevel', 'value': 101437.74}],
 'id': 6019977378,
 'timestamp': '2020-01-09 10:48:53',
 'sensor': {'sensor_type': {'name': 'BME280',
   'id': 17,
   'manufacturer': 'Bosch'},
  'id': 16150,
  'pin': '11'}}

Det finns olika typer av värden:

* P1 är PM10, alltså partiklar mindre än 10 mikrometer
* P2 är PM2.5, alltså partiklar mindre än 2,5 mikrometer
* temperature är temperatur
* humidity är luftfuktighet
* pressure är lufttryck